In [1]:
# Search_and_tf-idf
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %% [markdown]
# Here is my reworked search search function.

# %%
import re, glob, os, winsound
from collections import defaultdict
import pandas as pd

import winsound

frequency = 440  # Set Frequency To 2500 Hertz
duration = 2000  # Set Duration To 1000 ms == 1 second


# %%
# ver. 2 - uses a list of regex terms.

### NOTE ###
# - The code counts terms twice, if they are similar within the same pass. This means that drunk* and drun* count 1 and 2, because the loop counts drunk* again on the second iteration. THIS IS A PROBLEM WITH DRUNK* AND DRUNKARD!!! check https://stackoverflow.com/questions/1374457/find-out-how-many-times-a-regex-matches-in-a-string-in-python
# -  

def regex_search(folder_path , regList, verbose=1):
    """
    Search a folder path for a given regex.

    :folder_path: a string with the path to a dir with processed .txt files 
    :regList: a list of regular expressions as strings.
    """
    resultsDict = defaultdict(lambda: defaultdict(dict)) #consider using regular dict for the inner layer, since you know which terms you are searching.


    file_names = []

    # compile any search term(s) given for searching.
    #search_terms = re.compile('|'.join(regList)) # with word boundaries: re.compile(r'\b(?:%s)\b' % '|'.join(regList))
    search_terms_re = [re.compile(reg) for reg in regList]


    #loop through the folder with txt files.
    
    for filepath in glob.iglob(folder_path + "/*"):  #for future reference, if you just want to iterate through a number of files, use os.listdir() and slice it [:20] for instance.
        n_hits = 0
        filename = os.path.basename(filepath)
        #file_names.append(filename)
        
        #open the file
        with open(filepath, "r",  encoding = 'utf-8') as infile:
            content = infile.read()

        #find the document length (total tokens)
        resultsDict[filename]['text_length'] = len(re.findall(r'\w+', content))

        #search for the terms in the file, given the list of regexes.
    
        for term in search_terms_re:
            if term.findall(content):
                n_hits += 1
           

        # add the coutn of hits to the nested results dictionary 
            resultsDict[filename][term.pattern] = n_hits

    #export to df
    resultsDF = pd.DataFrame.from_dict(resultsDict, orient='index')
    # (re)name the index column
    #resultsDF = resultsDF.rename_axis('file_id')
      
    return resultsDF

#print(output)
#print(list(results.items())[:10])


#### for this code I used solutions from:
# https://stackoverflow.com/questions/6750240/how-to-do-re-compile-with-a-list-in-python
# https://howchoo.com/g/yjjknjdinmq/nested-defaultdict-python
# https://stackoverflow.com/questions/19851005/rename-pandas-dataframe-index 
# https://www.geeksforgeeks.org/python-program-to-count-words-in-a-sentence/


# %%
# searchQuery = [
#                 "drunk*",
#                 "intoxicat*",
#                 "temperance",
#                 "beer*",
#                 "liquor",
#                 "alcohol*",
#                 "brandy",
#                 "booz*",
#                 "drunkard",
#                 "by-drink*"]

searchQuery = [
                'poor', 
                'poverty', 
                'underclass', 
                'ragg*',
                'unemploy*']

# %%

#function call

output = regex_search("C:/Users/Stell/Documents/Digital Humanities/18" , searchQuery)


# %%
# inspect the dataframe
#output


# %%
# write the output to a human friendly files

# output.to_csv('results_xorb_18.tsv', sep="\t", index_label='file_id')
output.to_excel('results_stella_18.xlsx', index_label='file_id')

winsound.Beep(frequency, duration)
#winsound.MessageBeep()

# %% [markdown]
# Get the counts for tf-idf

# %%

df_results = pd.read_excel("results_stella_18.xlsx")


In [2]:
print(df_results)

                                        file_id  text_length  poor  poverty  \
0        1800-01-15_t18000115-87-verdict517.txt          420     0        0   
1         1800-04-02_t18000402-34-punish190.txt          231     0        0   
2         1800-04-02_t18000402-42-punish238.txt          742     0        0   
3          1800-05-28_t18000528-6-verdict37.txt         5002     0        0   
4        1800-07-09_t18000709-20-verdict108.txt          654     0        0   
..                                          ...          ...   ...      ...   
385      1898-05-16_t18980516-374-verdict-1.txt           20     0        0   
386   1898-11-21_t18981121-51-punishment-53.txt         1199     0        0   
387       1898-12-12_t18981212-65-verdict-1.txt           39     0        0   
388   1898-12-12_t18981212-88-punishment-24.txt           47     0        0   
389  1899-10-23_t18991023-659-punishment-27.txt         2198     0        0   

     underclass  ragg*  unemploy*  
0             0

In [2]:
df_results

,file_id,text_length,drunk*,intoxicat*,temperance,beer*,liquor,alcohol*,brandy,booz*,drunkard,by-drink*
0,1800-01-15_t18000115-87-verdict517.txt,420,0,0,0,1,1,1,1,1,1,1
1,1800-04-02_t18000402-34-punish190.txt,231,0,0,0,0,0,0,0,0,0,0
2,1800-04-02_t18000402-42-punish238.txt,742,0,0,0,0,0,0,0,0,0,0
3,1800-05-28_t18000528-6-verdict37.txt,5002,0,0,0,1,1,1,2,2,2,2
4,1800-07-09_t18000709-20-verdict108.txt,654,0,0,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
385,1898-05-16_t18980516-374-verdict-1.txt,20,0,0,0,0,0,0,0,0,0,0
386,1898-11-21_t18981121-51-punishment-53.txt,1199,1,1,1,2,2,2,2,2,2,2
387,1898-12-12_t18981212-65-verdict-1.txt,39,0,0,0,0,0,0,0,0,0,0
388,1898-12-12_t18981212-88-punishment-24.txt,47,0,0,0,0,0,0,0,0,0,0


tf = amount of times term occurs in doc / number of words in doc 

idf = Total number of documents / Number of documents with term in it

In [3]:
df_calc = pd.DataFrame()

In [4]:
df_calc['file_id'] = df_results['file_id']

In [5]:
df_calc

,file_id
0,1800-01-15_t18000115-87-verdict517.txt
1,1800-04-02_t18000402-34-punish190.txt
2,1800-04-02_t18000402-42-punish238.txt
3,1800-05-28_t18000528-6-verdict37.txt
4,1800-07-09_t18000709-20-verdict108.txt
...,...
385,1898-05-16_t18980516-374-verdict-1.txt
386,1898-11-21_t18981121-51-punishment-53.txt
387,1898-12-12_t18981212-65-verdict-1.txt
388,1898-12-12_t18981212-88-punishment-24.txt


In [6]:
df_calc['text_length'] = df_results['text_length']

In [7]:
df_calc

,file_id,text_length
0,1800-01-15_t18000115-87-verdict517.txt,420
1,1800-04-02_t18000402-34-punish190.txt,231
2,1800-04-02_t18000402-42-punish238.txt,742
3,1800-05-28_t18000528-6-verdict37.txt,5002
4,1800-07-09_t18000709-20-verdict108.txt,654
...,...,...
385,1898-05-16_t18980516-374-verdict-1.txt,20
386,1898-11-21_t18981121-51-punishment-53.txt,1199
387,1898-12-12_t18981212-65-verdict-1.txt,39
388,1898-12-12_t18981212-88-punishment-24.txt,47


In [14]:
df_calc['drunk_freq*'] = df_results['drunk*']


# for index, term in enumerate(df_results['drunk*']):
#     #for index2, term2 in enumerate(df_calc['file_id']):
#     df_calc.iloc[index]['tf_drunk*'] = (df_results.iloc[index]['drunk*'] / df_results.iloc[index]['text_length'])

In [15]:
df_calc

,file_id,text_length,tf_drunk*,drunk_freq*
0,1800-01-15_t18000115-87-verdict517.txt,420,0,0
1,1800-04-02_t18000402-34-punish190.txt,231,0,0
2,1800-04-02_t18000402-42-punish238.txt,742,0,0
3,1800-05-28_t18000528-6-verdict37.txt,5002,0,0
4,1800-07-09_t18000709-20-verdict108.txt,654,0,0
...,...,...,...,...
385,1898-05-16_t18980516-374-verdict-1.txt,20,0,0
386,1898-11-21_t18981121-51-punishment-53.txt,1199,0,1
387,1898-12-12_t18981212-65-verdict-1.txt,39,0,0
388,1898-12-12_t18981212-88-punishment-24.txt,47,0,0


In [18]:
df_calc['tf_drunk*'] = (df_calc['drunk_freq*'] / df_calc['text_length']) 

In [19]:
df_calc

,file_id,text_length,tf_drunk*,drunk_freq*
0,1800-01-15_t18000115-87-verdict517.txt,420,0.000000,0
1,1800-04-02_t18000402-34-punish190.txt,231,0.000000,0
2,1800-04-02_t18000402-42-punish238.txt,742,0.000000,0
3,1800-05-28_t18000528-6-verdict37.txt,5002,0.000000,0
4,1800-07-09_t18000709-20-verdict108.txt,654,0.000000,0
...,...,...,...,...
385,1898-05-16_t18980516-374-verdict-1.txt,20,0.000000,0
386,1898-11-21_t18981121-51-punishment-53.txt,1199,0.000834,1
387,1898-12-12_t18981212-65-verdict-1.txt,39,0.000000,0
388,1898-12-12_t18981212-88-punishment-24.txt,47,0.000000,0


In [20]:
print(len(df_calc['file_id']))

390


In [65]:
import math
counter = 0
for index, term in enumerate(df_calc['drunk_freq*']):
    if df_calc.iloc[index]['drunk_freq*'] != 0:
        counter+= 1
        
df_calc['idf_drunk*'] = (math.log((len(df_calc['file_id']))/ counter))

In [37]:
df_calc

,file_id,text_length,tf_drunk*,drunk_freq*,idf_drunk*,tf.idf_drunk*
0,1800-01-15_t18000115-87-verdict517.txt,420,0.000000,0,2.970414,0.000000
1,1800-04-02_t18000402-34-punish190.txt,231,0.000000,0,2.970414,0.000000
2,1800-04-02_t18000402-42-punish238.txt,742,0.000000,0,2.970414,0.000000
3,1800-05-28_t18000528-6-verdict37.txt,5002,0.000000,0,2.970414,0.000000
4,1800-07-09_t18000709-20-verdict108.txt,654,0.000000,0,2.970414,0.000000
...,...,...,...,...,...,...
385,1898-05-16_t18980516-374-verdict-1.txt,20,0.000000,0,2.970414,0.000000
386,1898-11-21_t18981121-51-punishment-53.txt,1199,0.000834,1,2.970414,0.016264
387,1898-12-12_t18981212-65-verdict-1.txt,39,0.000000,0,2.970414,0.000000
388,1898-12-12_t18981212-88-punishment-24.txt,47,0.000000,0,2.970414,0.000000


In [38]:
df_calc['tf.idf_drunk*'] = df_calc['tf_drunk*'] * df_calc['idf_drunk*']

In [39]:
df_calc

,file_id,text_length,tf_drunk*,drunk_freq*,idf_drunk*,tf.idf_drunk*
0,1800-01-15_t18000115-87-verdict517.txt,420,0.000000,0,2.970414,0.000000
1,1800-04-02_t18000402-34-punish190.txt,231,0.000000,0,2.970414,0.000000
2,1800-04-02_t18000402-42-punish238.txt,742,0.000000,0,2.970414,0.000000
3,1800-05-28_t18000528-6-verdict37.txt,5002,0.000000,0,2.970414,0.000000
4,1800-07-09_t18000709-20-verdict108.txt,654,0.000000,0,2.970414,0.000000
...,...,...,...,...,...,...
385,1898-05-16_t18980516-374-verdict-1.txt,20,0.000000,0,2.970414,0.000000
386,1898-11-21_t18981121-51-punishment-53.txt,1199,0.000834,1,2.970414,0.002477
387,1898-12-12_t18981212-65-verdict-1.txt,39,0.000000,0,2.970414,0.000000
388,1898-12-12_t18981212-88-punishment-24.txt,47,0.000000,0,2.970414,0.000000


file_id
text_length
tf_drunk*
drunk_freq*
idf_drunk*
tf.idf_drunk*


In [3]:
import math

def get_tf_idf(final_df, freq_df, term):
    """
    Blablabla
    """
    
    final_df[term+'_freq'] = freq_df[term]
    final_df['tf_'+term] = (final_df[str(term+'_freq')] / final_df['text_length'])
    
    counter = 0
    for index, freq in enumerate(final_df[term+'_freq']):
        if final_df.iloc[index][term+'_freq'] != 0:
            counter+= 1
        
    final_df['idf_'+term] = (math.log((len(final_df['file_id']))/ counter))
    
    final_df['tf.idf_'+term] = final_df['tf_'+term] * final_df['idf_'+term]
    
    columns = ['tf_'+term, 'idf_'+term, term+'_freq']
    final_df = final_df.drop(columns, axis=1)
    return(final_df)

In [4]:
final_df = pd.DataFrame()
final_df['file_id'] = df_results['file_id']
final_df['text_length'] = df_results['text_length']

freq_df = df_results

final_df = get_tf_idf(final_df, freq_df, 'poverty')
final_df = get_tf_idf(final_df, freq_df, 'poor')
final_df = get_tf_idf(final_df, freq_df, 'ragg*')
final_df = get_tf_idf(final_df, freq_df, 'unemploy*')


# final_df = get_tf_idf(final_df, freq_df, 'drunk*')
# final_df = get_tf_idf(final_df, freq_df, 'beer*')
# final_df = get_tf_idf(final_df, freq_df, 'intoxicat*')
# final_df = get_tf_idf(final_df, freq_df, 'temperance')
# final_df = get_tf_idf(final_df, freq_df, 'liquor')
# final_df = get_tf_idf(final_df, freq_df, 'alcohol*')
# final_df = get_tf_idf(final_df, freq_df, 'brandy')
# final_df = get_tf_idf(final_df, freq_df, 'booz*')
# final_df = get_tf_idf(final_df, freq_df, 'drunkard')
# final_df = get_tf_idf(final_df, freq_df, 'by-drink*')

In [5]:
final_df

,file_id,text_length,tf.idf_poverty,tf.idf_poor,tf.idf_ragg*,tf.idf_unemploy*
0,1800-01-15_t18000115-87-verdict517.txt,420,0.0,0.0,0.0,0.0
1,1800-04-02_t18000402-34-punish190.txt,231,0.0,0.0,0.0,0.0
2,1800-04-02_t18000402-42-punish238.txt,742,0.0,0.0,0.0,0.0
3,1800-05-28_t18000528-6-verdict37.txt,5002,0.0,0.0,0.0,0.0
4,1800-07-09_t18000709-20-verdict108.txt,654,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
385,1898-05-16_t18980516-374-verdict-1.txt,20,0.0,0.0,0.0,0.0
386,1898-11-21_t18981121-51-punishment-53.txt,1199,0.0,0.0,0.0,0.0
387,1898-12-12_t18981212-65-verdict-1.txt,39,0.0,0.0,0.0,0.0
388,1898-12-12_t18981212-88-punishment-24.txt,47,0.0,0.0,0.0,0.0


In [6]:
final_df.to_excel('final_results_stella_18.xlsx', index_label='file_id')